# 4) Exploratory Analysis

I envision that it should be possible to parse the SMILES structural codes into meaningful chemical information (molecular weight, hydrogen bond donors/acceptors, degree of branching, etc.) that is useful in predicting the compound's boiling point.

Before proceeding, let's evaluate that hypothesis by parsing a subset of the structural codes and examining the correlation coefficients.

*Note:* For copyright reasons, the CSV files containing the boiling point data are not included in this repository.

## Import the necessary modules & data.

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Lipinski, Descriptors
from numpy import nan

In [3]:
# Import the training sets.
x_train = pd.read_csv("data/x_train-CHNO.csv")
y_train = pd.read_csv("data/y_train-CHNO.csv")

## Create a combined data set.

In [4]:
# Merge the training sets into a combined data frame.
xy_train = x_train.copy()
xy_train["BP"] = y_train

In [5]:
x_train

,SMILES
0,CCCCCCC(C)CCC(C)C(O)=O
1,CCCCCCCCC(C)CCCCCCC=O
2,CC(C)c1ccc(C=O)cc1
3,c1ccc(cc1)N=Nc2ccccc2
4,CCCCCCCCCCC(C)CCCCC(C)C#C
...,...
7452,CCCC=CCC(C)CC(C)C=O
7453,CCCCCCCC=CCCCCCO
7454,CCCCCCCCCC=CCC(C)O
7455,CC(C)CCCCCCCCCC=CCC(O)=O


## Define a function for parsing SMILES codes.

First, use dictionaries to define functional groups and other structural features of interest. Then, use a function to append this interpreted structural information to `xy_train` as a series of new columns.

First, let's define a function to pull *a lot* of potential boiling point predictors out of the SMILES structural code. The `RDKit` module will do a lot of the heavy lifting here, and we'll only use a small portion of these in actually training the model later.

I selected these potential features based on knowledge of the underlying chemistry. I included potential features that I thought would correlate strongly to boiling point (e.g.: molecular weight, hydrogen bond donors), in addition to others that I expected to have a weaker correlation, if any at all.

In [6]:
# This dict contains functional groups defined by SMARTS substructure codes.
# The interpret_smiles() function will use these definitions to count the number
# of times each functional group occurs within a SMILES code.

descriptors1 = {
                "alcohols, any type" : Chem.MolFromSmarts("[#6;!$(C=O)][OX2H1]"),
                "alcohol, primary" : Chem.MolFromSmarts("[CH2]([#6])[OH1]"),
                "alcohol, secondary" : Chem.MolFromSmarts("[CH1]([#6])([#6])[OH1]"),
                "alcohol, tertiary" : Chem.MolFromSmarts("[CH0]([#6])([#6])([#6])[OH1]"),
                "alcohol, vinyl" : Chem.MolFromSmarts("[$([CX3]=[CX3])][OH1]"),
                "alcohol, aryl" : Chem.MolFromSmarts("a[OH1]"),
                "ethers" : Chem.MolFromSmarts("[OD2]([#6;!$(C=O)])[#6;!$(C=O)]"),
                "amines, primary" : Chem.MolFromSmarts("[#6][NX3;H2;!$(NC=O)]"),
                "amines, secondary" : Chem.MolFromSmarts("[#6][NX3;H1;!$(NC=O)]"),
                "amines, tertiary" : Chem.MolFromSmarts("[#6][NX3;H0;!$(NC=O)]"),
                "carbonyl, all types" : Chem.MolFromSmarts("[CX3]=[OX1]"),
                "carbonyl, ester" : Chem.MolFromSmarts("[#6][CX3](=[OX1])O[C,c]"),
                "carbonyl, acid " : Chem.MolFromSmarts("[#6][CX3](=[OX1])[OH]"),
                "carbonyl, all amides" : Chem.MolFromSmarts("[#6][CX3](=[OX1])[NX3]"),
                "carbonyl, primary amides" : Chem.MolFromSmarts("[#6][CX3](=[OX1])[NX3H2]"),
                "carbonyl, secondary amides" : Chem.MolFromSmarts("[#6][CX3](=[OX1])[NX3H1]"),
                "carbonyl, tertiary amides" : Chem.MolFromSmarts("[#6][CX3](=[OX1])[NX3H0]"),
                "carbonyl, aldehyde" : Chem.MolFromSmarts("[#6][CH1X3](=[OX1])"),
                "carbonyl, ketone" : Chem.MolFromSmarts("[#6][CX3](=[OX1])[#6]"),
                "imine" : Chem.MolFromSmarts("[$([CX3]([#6])[#6]),$([CX3H][#6])]=[$([NX2][#6]),$([NX2H])]"),
                "nitrile" : Chem.MolFromSmarts("[NX1]#[CX2]"),
                "olefins, all types" : Chem.MolFromSmarts("[CX3]=[CX3]"),
                "alkynes, all types" : Chem.MolFromSmarts("[CX2]#[CX2]"),
                "carbon atoms, all" : Chem.MolFromSmarts("[#6]"),
                "carbon atoms, all aliphatic" : Chem.MolFromSmarts("[CX4]"),
                "carbon atoms, aliphatic branches" : Chem.MolFromSmarts("[$([C;H1,H0]([#6])([#6])([#6]))]"),
                "halides, total" : Chem.MolFromSmarts("[F,Cl,Br,I]"),
                "halides, bromide" : Chem.MolFromSmarts("[Br]"),
                "halides, iodide" : Chem.MolFromSmarts("[I]"),
                }

In [7]:
# This dict contains other structural information that may be useful in
# estimating boiling points. These descriptors do _not_ use SMARTS substructures.
# The interpret_smiles() function will use these definitions to generate
# additional substructure information from a SMILES code.

descriptors2 = {
                "rings, aromatic" : Lipinski.NumAromaticRings,
                "rings, saturated" : Lipinski.NumAliphaticRings,
                "H-bond acceptors" : Lipinski.NumHAcceptors,
                "H-Bond donors" : Lipinski.NumHDonors,
                "heavy atom count" : Lipinski.HeavyAtomCount,
                "molecular weight" : Descriptors.ExactMolWt
                }

In [8]:
# This function will accept a single row from the bp_data dataframe.

def interpret_smiles(row):
    
    # Get the SMILES code, then use it to create an RDKit mol object.
    smiles_code = row["SMILES"]
    molecule = Chem.MolFromSmiles(smiles_code)
    
    # If the SMILES code was properly interpreted, compute the descriptors of interest.
    # If not, populate the columns with NaN instead.
    
    if molecule is not None:

        # Add new columns with counts of each of the functional groups defined in
        # the descriptors1 dict.
        for property in descriptors1.keys():
            row[property] = len(molecule.GetSubstructMatches(descriptors1[property]))

        # Add new columns with counts of each of the properties defined in
        # the descriptors2 dict.
        for property in descriptors2.keys():
            row[property] = descriptors2[property](molecule)
            
    else:
        for property in descriptors1.keys():
            row[property] = nan
        for property in descriptors2.keys():
            row[property] = nan
        
    return row

## Parse the SMILES codes.
Actually use the `interpret_smiles()` function defined above.

In [9]:
xy_train = xy_train.apply(interpret_smiles, axis = 1)

How many SMILES codes couldn't be interpreted?

In [10]:
xy_train[xy_train.isna().any(axis = 1)]

,SMILES,BP,"alcohols, any type","alcohol, primary","alcohol, secondary","alcohol, tertiary","alcohol, vinyl","alcohol, aryl",ethers,"amines, primary",...,"carbon atoms, aliphatic branches","halides, total","halides, bromide","halides, iodide","rings, aromatic","rings, saturated",H-bond acceptors,H-Bond donors,heavy atom count,molecular weight


Nice! Only one SMILES code failed here.

Let's add a couple secondary parameters, calculated from some of the results returned by `RDKit`: the fraction of carbon atoms that are aliphatic, and the degree of branching (a ratio of branched carbons to total carbon atoms).

In [11]:
xy_train["aliphatic fraction"] = xy_train["carbon atoms, all aliphatic"] / xy_train["carbon atoms, all"]

In [12]:
def calc_branch_frac(row):
    if row["carbon atoms, all aliphatic"] != 0:
        row["branching fraction"] = row["carbon atoms, aliphatic branches"] / row["carbon atoms, all"]
    else:
        row["branching fraction"] = nan
        
    return row

In [13]:
xy_train = xy_train.apply(calc_branch_frac, axis = 1)

In [15]:
xy_train.head()

,SMILES,BP,"alcohols, any type","alcohol, primary","alcohol, secondary","alcohol, tertiary","alcohol, vinyl","alcohol, aryl",ethers,"amines, primary",...,"halides, bromide","halides, iodide","rings, aromatic","rings, saturated",H-bond acceptors,H-Bond donors,heavy atom count,molecular weight,aliphatic fraction,branching fraction
0,CCCCCCC(C)CCC(C)C(O)=O,580.59,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,15,214.193280,0.923077,0.153846
1,CCCCCCCCC(C)CCCCCCC=O,600.82,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,18,254.260966,0.941176,0.058824
2,CC(C)c1ccc(C=O)cc1,508.70,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,11,148.088815,0.300000,0.100000
3,c1ccc(cc1)N=Nc2ccccc2,566.20,0,0,0,0,0,0,0,0,...,0,0,2,0,2,0,14,182.084398,0.000000,NaN
4,CCCCCCCCCCC(C)CCCCC(C)C#C,614.48,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,20,278.297351,0.900000,0.100000


## Examine correlation coefficients.
Determine the Spearman correlation coefficients to evaluate which of these structural components might be good features to use when we're ready to train the model.

In [16]:
pd.options.display.max_rows, pd.options.display.max_columns = None, None
xy_train.corr(method = "spearman")

,BP,"alcohols, any type","alcohol, primary","alcohol, secondary","alcohol, tertiary","alcohol, vinyl","alcohol, aryl",ethers,"amines, primary","amines, secondary","amines, tertiary","carbonyl, all types","carbonyl, ester","carbonyl, acid","carbonyl, all amides","carbonyl, primary amides","carbonyl, secondary amides","carbonyl, tertiary amides","carbonyl, aldehyde","carbonyl, ketone",imine,nitrile,"olefins, all types","alkynes, all types","carbon atoms, all","carbon atoms, all aliphatic","carbon atoms, aliphatic branches","halides, total","halides, bromide","halides, iodide","rings, aromatic","rings, saturated",H-bond acceptors,H-Bond donors,heavy atom count,molecular weight,aliphatic fraction,branching fraction
BP,1.000000,-0.063395,0.019994,-0.046804,-0.126879,NaN,-0.053959,-0.107947,0.012601,-0.005797,-0.044162,0.296261,-0.144609,0.494219,-0.006228,NaN,NaN,-0.006228,0.059570,0.071059,-0.010244,0.027445,0.210117,0.060633,0.830157,0.606527,-0.192261,NaN,NaN,NaN,-0.031722,-0.065248,0.103884,0.329825,0.884498,0.858353,-0.113546,-0.424952
"alcohols, any type",-0.063395,1.000000,0.759630,0.435751,0.273261,NaN,0.288648,-0.053514,-0.033396,-0.032686,-0.037102,-0.360283,-0.144434,-0.151766,-0.006288,NaN,NaN,-0.006288,-0.152740,-0.086890,-0.004446,-0.019404,0.074854,-0.129524,-0.219729,-0.136305,-0.050575,NaN,NaN,NaN,-0.005821,-0.037561,0.140445,0.608185,-0.210418,-0.180920,0.094037,0.031532
"alcohol, primary",0.019994,0.759630,1.000000,-0.048808,-0.025764,NaN,-0.032249,-0.042788,-0.025380,-0.028402,-0.028196,-0.276320,-0.110798,-0.116472,-0.004779,NaN,NaN,-0.004779,-0.116077,-0.067421,-0.003379,-0.014746,0.175932,-0.099430,-0.118759,-0.060544,-0.028587,NaN,NaN,NaN,-0.097284,-0.052742,0.104310,0.461647,-0.115825,-0.087871,0.046250,0.026373
"alcohol, secondary",-0.046804,0.435751,-0.048808,1.000000,-0.017530,NaN,-0.018490,-0.024533,-0.014551,-0.007509,-0.016167,-0.154291,-0.063526,-0.063984,-0.002740,NaN,NaN,-0.002740,-0.066553,-0.034900,-0.001937,-0.008455,-0.047800,-0.057008,-0.089733,-0.016066,-0.092731,NaN,NaN,NaN,-0.047401,0.021224,0.061785,0.265043,-0.083979,-0.066312,0.157149,-0.054253
"alcohol, tertiary",-0.126879,0.273261,-0.025764,-0.017530,1.000000,NaN,-0.011583,-0.015368,-0.009115,-0.010201,-0.010127,-0.095431,-0.035407,-0.041833,-0.001716,NaN,NaN,-0.001716,-0.041691,-0.024215,-0.001214,-0.005296,-0.056820,-0.031480,-0.153810,-0.084377,0.117712,NaN,NaN,NaN,-0.026726,0.003095,0.040855,0.166511,-0.145879,-0.133248,0.114969,0.145717
"alcohol, vinyl",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"alcohol, aryl",-0.053959,0.288648,-0.032249,-0.018490,-0.011583,NaN,1.000000,-0.007663,-0.009614,-0.010759,-0.010682,-0.104677,-0.041973,-0.044122,-0.001810,NaN,NaN,-0.001810,-0.043973,-0.025541,-0.001280,-0.005586,-0.079242,-0.037666,-0.102893,-0.164839,-0.056577,NaN,NaN,NaN,0.316463,-0.019980,0.043906,0.176331,-0.097449,-0.113823,-0.166734,-0.024578
ethers,-0.107947,-0.053514,-0.042788,-0.024533,-0.015368,NaN,-0.007663,1.000000,-0.012757,-0.014276,-0.004503,-0.131390,-0.050074,-0.058542,0.055797,NaN,NaN,0.055797,-0.058344,-0.029633,-0.001698,-0.007412,-0.092925,-0.049976,-0.064627,-0.002810,-0.199092,NaN,NaN,NaN,0.010598,-0.005198,0.065241,-0.090461,-0.057533,-0.039076,0.145950,-0.184117
"amines, primary",0.012601,-0.033396,-0.025380,-0.014551,-0.009115,NaN,-0.009614,-0.012757,1.000000,0.024009,0.040276,-0.082380,-0.033033,-0.034724,-0.001425,NaN,NaN,-0.001425,-0.034607,-0.020100,-0.001007,-0.004396,-0.064371,-0.029643,-0.050637,-0.072292,-0.087941,NaN,NaN,NaN,0.174247,0.001911,0.053143,0.140495,-0.038875,-0.049244,-0.061501,-0.058738
"amines, secondary",-0.005797,-0.032686,-0.028402,-0.007509,-0.010201,NaN,-0.010759,-0.014276,0.024009,1.000000,-0.009407,-0.088098,-0.036967,-0.034267,-0.001594,NaN,NaN,-0.001594,-0.038728,-0.022494,-0.001127,-0.004920,-0.069148,-0.033174,-0.019368,-0.003008,-0.094006,NaN,NaN,NaN,0.053211,0.045809,

## Wrapping up.

There are some attractive-looking candidates here. Strong correlations to heavy atom count, total carbon atoms, & molecular weight are not very surprising. All of these are closely related to one another, so we'll just use one of them as a feature -- **molecular weight**, as it's the most readily interpreted by chemists. It's worth noting that the number of halides present (total, only bromides, and only iodides) correlate with boiling point, too; halide features are redundant with molecular weight, but not the heavy atom count.

The fraction of aliphatic carbon atoms that are branch points, **branching fraction**, is also strongly correlated to boiling point and will make a valuable feature.

Finally, we'll round out an initial set of 3 features with the number of **hydrogen bond donors**. These three features are the most strongly correlated to boiling point, with lesser impacts coming from specific functional groups.